In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
%matplotlib inline

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
#load the dataset
#from google.colab import drive
#drive.mount('/gdrive')
#load the dataset
retail_df = pd.read_excel("Online Retail.xlsx")

## Cleanup

In [3]:
#remove additional spaces
retail_df['Description'] = retail_df['Description'].str.strip()
#remove NA values
retail_df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
#remove cancelled orders
retail_df['InvoiceNo'] = retail_df['InvoiceNo'].astype('str')
retail_df = retail_df[~retail_df['InvoiceNo'].str.contains('C')]

In [4]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

def create_basket(country_filter):
    basket = (retail_df[retail_df['Country'] == country_filter]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
    return basket
    

In [5]:
country_filter = "France"
basket_french = create_basket("France")
basket_sets = basket_french.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [6]:
basket_sets

Description  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  \
InvoiceNo                                                         
536370                            0                           0   
536852                            0                           0   
536974                            0                           0   
537065                            0                           0   
537463                            0                           0   
...                             ...                         ...   
580986                            0                           0   
581001                            0                           0   
581171                            0                           0   
581279                            0                           0   
581587                            0                           0   

Description  12 EGG HOUSE PAINTED WOOD  12 MESSAGE CARDS WITH ENVELOPES  \
InvoiceNo                                                                 
536370                               0                                0   
536852                               0                                0   
536974                               0                                0   
537065                               0                                0   
537463                               0                                0   
...                                ...                              ...   
580986                               0                                0   
581001                               0                                0   
581171                               0                                0   
581279                               0                                0   
581587                               0                                0   

Description  12 PENCIL SMALL TUBE WOODLAND  \
InvoiceNo                                    
536370                                   0   
536852                                   0   
536974                                   0   
537065                                   0   
537463                                   0   
...                                    ...   
580986                                   0   
581001                                   0   
581171                                   0   
581279                                   0   
581587                                   0   

Description  12 PENCILS SMALL TUBE RED RETROSPOT  12 PENCILS SMALL TUBE SKULL  \
InvoiceNo                                                                       
536370                                         0                            0   
536852                                         0                            0   
536974                                         0                            0   
537065                                         0                            0   
537463                                         0                            0   
...                                          ...                          ...   
580986                                         0                            0   
581001                                         0                            0   
581171                                         0                            0   
581279                                         0                            0   
581587                                         0                            0   

Description  12 PENCILS TALL TUBE POSY  12 PENCILS TALL TUBE RED RETROSPOT  \
InvoiceNo                                                                    
536370                               0                                   0   
536852                               0                                   0   
536974                               0                                   0   
537065                               0                                   0   
537463                         

In [7]:
frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)

In [23]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules.head()

antecedents                   consequents  \
0   (ALARM CLOCK BAKELIKE PINK)  (ALARM CLOCK BAKELIKE GREEN)   
1  (ALARM CLOCK BAKELIKE GREEN)   (ALARM CLOCK BAKELIKE PINK)   
2    (ALARM CLOCK BAKELIKE RED)  (ALARM CLOCK BAKELIKE GREEN)   
3  (ALARM CLOCK BAKELIKE GREEN)    (ALARM CLOCK BAKELIKE RED)   
4    (ALARM CLOCK BAKELIKE RED)   (ALARM CLOCK BAKELIKE PINK)   

   antecedent support  consequent support   support  confidence      lift  \
0            0.102041            0.096939  0.073980    0.725000  7.478947   
1            0.096939            0.102041  0.073980    0.763158  7.478947   
2            0.094388            0.096939  0.079082    0.837838  8.642959   
3            0.096939            0.094388  0.079082    0.815789  8.642959   
4            0.094388            0.102041  0.073980    0.783784  7.681081   

   leverage  conviction  
0  0.064088    3.283859  
1  0.064088    3.791383  
2  0.069932    5.568878  
3  0.069932    4.916181  
4  0.064348    4.153061

According to the results a customer is 7 times more likely to buy a pink alarm clock bakelike than an average customer (lift) if he/she buys ALARM CLOCK BAKELIKE GREEN. This rule is "true" in 77% of the cases (confidence). This can be used as insight to recommend **ALARM CLOCK BAKELIKE PINK** for tthose who bought the Green one.

In [15]:
basket_germany = create_basket("Germany")
basket2_sets = basket_germany.applymap(encode_units)
basket2_sets.drop('POSTAGE', inplace=True, axis=1)

In [16]:
frequent_itemsets_germany = apriori(basket2_sets, min_support=0.05, use_colnames=True)

In [17]:
rules = association_rules(frequent_itemsets_germany, metric="lift", min_threshold=1.2)
rules.head()

antecedents                          consequents  \
0    (PLASTERS IN TIN WOODLAND ANIMALS)      (PLASTERS IN TIN CIRCUS PARADE)   
1       (PLASTERS IN TIN CIRCUS PARADE)   (PLASTERS IN TIN WOODLAND ANIMALS)   
2   (ROUND SNACK BOXES SET OF 4 FRUITS)      (PLASTERS IN TIN CIRCUS PARADE)   
3       (PLASTERS IN TIN CIRCUS PARADE)  (ROUND SNACK BOXES SET OF 4 FRUITS)   
4  (ROUND SNACK BOXES SET OF4 WOODLAND)      (PLASTERS IN TIN CIRCUS PARADE)   

   antecedent support  consequent support   support  confidence      lift  \
0            0.137856            0.115974  0.067834    0.492063  4.242887   
1            0.115974            0.137856  0.067834    0.584906  4.242887   
2            0.157549            0.115974  0.050328    0.319444  2.754455   
3            0.115974            0.157549  0.050328    0.433962  2.754455   
4            0.245077            0.115974  0.056893    0.232143  2.001685   

   leverage  conviction  
0  0.051846    1.740427  
1  0.051846    2.076984  
2  0.032057    1.298977  
3  0.032057    1.488330  
4  0.028470    1.151290

According to the results a customer is 4 times more likely to buy PLASTERS IN TIN WOODLAND ANIMALS than an average customer (lift) if he/she buys PLASTERS IN TIN CIRCUS PARADE. This rule is "true" in 42% of the cases (confidence). This can be used as insight to recommend PLASTERS IN TIN WOODLAND ANIMALS for tthose who bought the Green one.

At this point, you may want to look at how much opportunity there is to use the popularity of one product to drive sales of another.

In [18]:
basket_uk = create_basket("United Kingdom")
basket3_sets = basket_uk.applymap(encode_units)
basket3_sets.drop('POSTAGE', inplace=True, axis=1)

In [19]:
frequent_itemsets_uk = apriori(basket3_sets, min_support=0.05, use_colnames=True)
rules = association_rules(frequent_itemsets_uk, metric="lift", min_threshold=0.8)
rules.head()

Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []

As we didn't find any rules for United Kigdom customers. We will try other filters.

In [20]:
retail_filtered = retail_df[retail_df['Country']=="United Kingdom"]

In [0]:
basket = (retail_filtered[retail_filtered['Quantity']<10]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [0]:
basket4_sets = basket.applymap(encode_units)
basket4_sets.drop('POSTAGE', inplace=True, axis=1)

In [17]:
frequent_itemsets_uk = apriori(basket3_sets, min_support=0.03, use_colnames=True)
rules = association_rules(frequent_itemsets_uk, metric="lift", min_threshold=0.5)
rules.head()

antecedents  ... conviction
0         (ALARM CLOCK BAKELIKE RED)  ...   2.415142
1       (ALARM CLOCK BAKELIKE GREEN)  ...   2.659288
2   (PINK REGENCY TEACUP AND SAUCER)  ...   5.300203
3  (GREEN REGENCY TEACUP AND SAUCER)  ...   2.517719
4  (ROSES REGENCY TEACUP AND SAUCER)  ...   3.551237

[5 rows x 9 columns]

## Conclusion - Perspective

We arrived to distinguich some rules related to France and Germany Market which can be used to make recommendations for our customers, but to better understand their preferences. I would suggest if I have more time that I categorize the products by finding their root words, like for example for ALARM CLOCK BAKELIKE GREEN, the root word can be Alarm (more general category) and use these categories to cluster the product purchases.

Also, that will be interesting to apply Collaborative Filtering Recommendations based on preferred items and items bought together.